# TIPI Scoring

In [23]:
import os, sys
import pandas as pd
import numpy as np
from scipy import stats
import pylab as plt

In [24]:
from analysis.CodingScoring import scoreTIPI, normTIPI

Import data


In [4]:
questions = pd.read_csv('dataHeadings.csv')
dataRaw = pd.read_csv('dataRaw.csv', names=questions.QuestionCode)

## Get TIPI scores

Code

In [5]:
dataTIPIRaw = dataRaw[['E+','A-','C+','S-','O+','E-','A+','C-','S+','O-']].dropna()

dataTIPI = dataTIPIRaw.replace(['Disagree strongly','Disagree moderately','Disagree slightly',
                                'Neither agree nor disagree','Agree slightly','Agree moderately','Agree strongly'],
                               [1,2,3,4,5,6,7])


Reverse code

In [6]:
dataTIPI[['A-','S-','E-','C-','O-']] = 8-dataTIPI.filter(regex='-')

Check

In [7]:
dataTIPIRaw.head()

,E+,A-,C+,S-,O+,E-,A+,C-,S+,O-
0,Agree moderately,Agree strongly,Neither agree nor disagree,Agree slightly,Agree strongly,Agree slightly,Agree strongly,Agree moderately,Agree slightly,Disagree moderately
1,Agree slightly,Agree moderately,Agree strongly,Agree slightly,Agree slightly,Disagree slightly,Agree moderately,Disagree strongly,Agree moderately,Disagree strongly
2,Agree slightly,Agree moderately,Agree slightly,Agree slightly,Agree moderately,Agree slightly,Agree slightly,Agree slightly,Agree slightly,Agree slightly
3,Disagree slightly,Agree moderately,Agree moderately,Disagree moderately,Agree moderately,Disagree slightly,Disagree slightly,Disagree moderately,Agree moderately,Neither agree nor disagree
4,Disagree moderately,Disagree slightly,Agree slightly,Agree slightly,Neither agree nor disagree,Agree slightly,Agree moderately,Neither agree nor disagree,Agree slightly,Agree slightly


In [8]:
dataTIPI.head()

,E+,A-,C+,S-,O+,E-,A+,C-,S+,O-
0,6,1,4,3,7,3,7,2,5,6
1,5,2,7,3,5,5,6,7,6,7
2,5,2,5,3,6,3,5,3,5,3
3,3,2,6,6,6,5,3,6,6,4
4,2,5,5,3,4,3,6,4,5,3


Score

In [9]:
TIPI_scored = pd.DataFrame()
TIPI_scored['O'] = (dataTIPI['O+']+dataTIPI['O-'])/2
TIPI_scored['C'] = (dataTIPI['C+']+dataTIPI['C-'])/2
TIPI_scored['E'] = (dataTIPI['E+']+dataTIPI['E-'])/2
TIPI_scored['A'] = (dataTIPI['A+']+dataTIPI['A-'])/2
TIPI_scored['S'] = (dataTIPI['S+']+dataTIPI['S-'])/2
TIPI_scored.head()

,O,C,E,A,S
0,6.5,3.0,4.5,4.0,4.0
1,6.0,7.0,5.0,4.0,4.5
2,4.5,4.0,4.0,3.5,4.0
3,5.0,6.0,4.0,2.5,6.0
4,3.5,4.5,2.5,5.5,4.0


In [10]:
TIPI_scored.describe()

,O,C,E,A,S
count,16181.000000,16181.000000,16181.000000,16181.000000,16181.000000
mean,5.295686,4.957079,3.600952,4.813454,4.510413
std,1.118450,1.364139,1.682553,1.224232,1.500494
min,1.000000,1.000000,1.000000,1.000000,1.000000
25%,4.500000,4.000000,2.000000,4.000000,3.500000
50%,5.500000,5.000000,3.500000,5.000000,4.500000
75%,6.000000,6.000000,5.000000,6.000000,6.000000
max,7.000000,7.000000,7.000000,7.000000,7.000000


In [11]:
TIPI_scored.corr()

,O,C,E,A,S
O,1.000000,-0.043542,0.240957,0.147523,0.096411
C,-0.043542,1.000000,-0.002609,0.086653,0.241843
E,0.240957,-0.002609,1.000000,0.061674,0.068337
A,0.147523,0.086653,0.061674,1.000000,0.214427
S,0.096411,0.241843,0.068337,0.214427,1.000000


In [12]:
np.save('TIPI_scored_OCEAS.npy',TIPI_scored)

In [13]:
TIPI_scored.head()

,O,C,E,A,S
0,6.5,3.0,4.5,4.0,4.0
1,6.0,7.0,5.0,4.0,4.5
2,4.5,4.0,4.0,3.5,4.0
3,5.0,6.0,4.0,2.5,6.0
4,3.5,4.5,2.5,5.5,4.0


In [14]:

TIPI_scored['N'] = 8-TIPI_scored['S']
TIPI_scored_NEOAC = TIPI_scored[['N','E','O','A','C']]
TIPI_scored_NEOAC.head()

,N,E,O,A,C
0,4.0,4.5,6.5,4.0,3.0
1,3.5,5.0,6.0,4.0,7.0
2,4.0,4.0,4.5,3.5,4.0
3,2.0,4.0,5.0,2.5,6.0
4,4.0,2.5,3.5,5.5,4.5


In [15]:
np.save('TIPI_scored_NEOAC.npy',TIPI_scored_NEOAC)

## Normalizing

In [16]:
TIPInorms = pd.read_csv('TIPINorms.csv', index_col=0)

In [17]:
arrTIPInorms = TIPInorms[['Openness','Conscientiousness','Extraversion','Agreeableness','Emotional Stability']].values
arrTIPInorms

array([[5.5078e+00, 4.6454e+00, 3.9506e+00, 4.7099e+00, 4.3579e+00],
       [1.1390e+00, 1.4146e+00, 1.5831e+00, 1.2261e+00, 1.4789e+00],
       [2.7800e+05, 2.7800e+05, 2.7800e+05, 2.7800e+05, 2.7800e+05]])

In [3]:
arrTIPI = np.load('data/TIPI_scored_OCEAS.npy')
arrTIPI

array([[6.5, 3. , 4.5, 4. , 4. ],
       [6. , 7. , 5. , 4. , 4.5],
       [4.5, 4. , 4. , 3.5, 4. ],
       ...,
       [5.5, 7. , 1. , 5.5, 6. ],
       [5. , 4. , 2.5, 4.5, 1. ],
       [4.5, 4. , 4. , 2. , 2.5]])

Norm according to TIPI norms because this sample is significantly different from the norms (t test, P<.001)

In [11]:
N_,D_ = np.shape(arrTIPI)
for d in range(D_):
    x_tmp = arrTIPI[:,d]
    x_mu = arrTIPInorms[0][d]
    x_std = arrTIPInorms[1][d]
    arrTIPI[:,d] = (x_tmp - x_mu)/x_std

NameError: name 'arrTIPInorms' is not defined

Reverse emotional stability into neuroticism

In [25]:
arrTIPI[:,4] = 0-arrTIPI[:,4]

In [26]:
arrTIPI

array([[ 0.87111501, -1.16315566,  0.34704062, -0.57899029,  0.24200419],
       [ 0.43213345,  1.6644988 ,  0.66287663, -0.57899029, -0.09608493],
       [-0.88481124, -0.45624205,  0.0312046 , -0.98678737,  0.24200419],
       ...,
       [-0.00684811,  1.6644988 , -1.86381151,  0.64440095, -1.11035229],
       [-0.44582968, -0.45624205, -0.91630346, -0.17119321,  2.27053891],
       [-0.88481124, -0.45624205,  0.0312046 , -2.21017862,  1.25627155]])

In [27]:
np.save('TIPI_scored_normed_OCEAN.npy',arrTIPI)

## Final norm method

In [25]:
df = pd.DataFrame(arrTIPI,columns=['O','C','E','A','S'])

In [26]:
df_TIPInormed = normTIPI(df)

In [27]:
df_TIPInormed.head()

,O,C,E,A,S
0,0.871115,-1.163156,0.347041,-0.578990,-0.242004
1,0.432133,1.664499,0.662877,-0.578990,0.096085
2,-0.884811,-0.456242,0.031205,-0.986787,-0.242004
3,-0.445830,0.957585,0.031205,-1.802382,1.110352
4,-1.762774,-0.102785,-0.916303,0.644401,-0.242004


In [28]:
arr_TIPInormed = df_TIPInormed[['S','E','O','A','C']].values

In [29]:
arr_TIPInormed[:,0] = 0-arr_TIPInormed[:,0]

In [30]:
arr_TIPInormed

array([[ 0.24200419,  0.34704062,  0.87111501, -0.57899029, -1.16315566],
       [-0.09608493,  0.66287663,  0.43213345, -0.57899029,  1.6644988 ],
       [ 0.24200419,  0.0312046 , -0.88481124, -0.98678737, -0.45624205],
       ...,
       [-1.11035229, -1.86381151, -0.00684811,  0.64440095,  1.6644988 ],
       [ 2.27053891, -0.91630346, -0.44582968, -0.17119321, -0.45624205],
       [ 1.25627155,  0.0312046 , -0.88481124, -2.21017862, -0.45624205]])

In [35]:
np.save('data/TIPI_scored_normed_NEOAC.npy',arr_TIPInormed)

In [36]:
arrTIPI = np.load('data/TIPI_scored_normed_NEOAC.npy')

In [37]:
arrTIPI

array([[ 0.24200419,  0.34704062,  0.87111501, -0.57899029, -1.16315566],
       [-0.09608493,  0.66287663,  0.43213345, -0.57899029,  1.6644988 ],
       [ 0.24200419,  0.0312046 , -0.88481124, -0.98678737, -0.45624205],
       ...,
       [-1.11035229, -1.86381151, -0.00684811,  0.64440095,  1.6644988 ],
       [ 2.27053891, -0.91630346, -0.44582968, -0.17119321, -0.45624205],
       [ 1.25627155,  0.0312046 , -0.88481124, -2.21017862, -0.45624205]])